<a href="https://colab.research.google.com/github/peterbmob/niteroi/blob/main/Tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CCS+Q - Molten NaCl revisited

In this tutoral, we going to revisit a classic example, namely molten NaCl. This has been studied extensively in the literature. Recently, machinel learning potentials were used. Can CCS compete?

First, we install ´ccs´ and dowload the DFT dataset, here in the form of an ASE database. Then we load the necessary modules.

In [ ]:
import seaborn as sns

In [ ]:
%%capture
!pip install ccs_fit
!pip install pymatgen
!wget https://github.com/peterbmob/niteroi/raw/main/DFT.db

In [ ]:
!ase db DFT.db

In [ ]:
# Load in the relevant packages
import os
from ase.io import read,write
from ase.build import bulk
import numpy as np
import ase.db as db
from ase.visualize import view
from ase.calculators.lj import LennardJones
import matplotlib.pyplot as plt
import pandas as pd
from ase.geometry.analysis import Analysis
base=os.getcwd()
print('base is: ', os.getcwd())
Fit_on_forces=True  #Enable/disable option for fitting CCS potential to atomic force

# Look at the data

In [ ]:
data=db.connect('DFT.db')

energy=[]; fmax=[]; formula=[]; BL=[]; volume=[]
df=pd.DataFrame()
for row in data.select():
  atoms=row.toatoms()
  formula.append(row.formula)
  energy.append(row.energy/len(atoms))
  fmax.append(row.fmax)
  volume.append(row.volume)
  ana = Analysis(atoms)
  Bonds = ana.get_bonds('Na', 'Cl', unique=True)
  BondValues = ana.get_values(Bonds)
  BL.append(np.average(BondValues))


df['formula']=formula
df['energy']=energy
df['fmax']=fmax
df['volume']=volume
df['BL']=BL
df
sns.pairplot(df)


In [ ]:
df['volume'].unique()

# Let us start with a smaller subset of the data.
To speed up the finding of the switching points, we start with a smaller subset of the data. Here, I want some data form the higher volume fraction and some from the lower ones.

In [ ]:
os.mkdir('potential')
os.chdir('potential')

In [ ]:
nu=pd.DataFrame({})
groups=['low', 'high']

for group in groups:
    if group == 'low':
        nums=np.random.randint(1,len(df.loc[df['volume']<2000]),50)
        nu[group]=nums
    if group == 'high':
        nums=np.random.randint(1,len(df.loc[df['volume']>2000]),50)
        nu[group]=nums

In [ ]:
dw=db.connect('DFT_train.db')

In [ ]:
energy=[]; fmax=[]; formula=[]; BL=[]; volume=[]; ok=[]
data_train=pd.DataFrame({})

for group in groups:
    values=nu[group].sort_values()
    counter = 0
    if group == 'low':
        for row in data.select('volume>2000'):
            counter=counter+1
            if counter in values:
                atoms=row.toatoms()
                formula.append(row.formula)
                energy.append(row.energy/len(atoms))
                fmax.append(row.fmax)
                volume.append(row.volume)
                ana = Analysis(atoms)
                Bonds = ana.get_bonds('Na', 'Cl', unique=True)
                BondValues = ana.get_values(Bonds)
                BL.append(np.average(BondValues))
                dw.write(atoms)
    if group == 'high':
        for row in data.select('volume<2000'):
            counter=counter+1
            if counter in values:
                atoms=row.toatoms()
                formula.append(row.formula)
                energy.append(row.energy/len(atoms))
                fmax.append(row.fmax)
                volume.append(row.volume)
                ana = Analysis(atoms)
                Bonds = ana.get_bonds('Na', 'Cl', unique=True)
                BondValues = ana.get_values(Bonds)
                BL.append(np.average(BondValues))
                dw.write(atoms)

data_train['formula']=formula
data_train['energy']=energy
data_train['fmax']=fmax
data_train['volume']=volume
data_train['BL']=BL

sns.pairplot(data_train)
print(len(data_train))


In [ ]:
!ase db DFT_train.db

# Fetch the data using ccs_fetch

In [ ]:
from ccs_fit.scripts.ccs_fetch import ccs_fetch

q={"Na":1.0,"Cl":-1.0}
ccs_fetch(mode='CCS+Q', DFT_DB='DFT_train.db', charge_dict=q, include_forces=Fit_on_forces)


# Generate input
Here in the first attempt, we use a large value for `Resolution` to speed up thesearch for the switching point.

In [ ]:
### Generate input.json file
import json

input={
    "General": {
        "interface": "CCS+Q",
        "merging"  : "True",
    },
    "Twobody": {
                "Cl-Na": {
                        "Rcut": 5.5,
                        "Resolution": 0.25,
                        "Swtype": "sw",
                        "const_type" : "Mono"
                },
                "Cl-Cl": {
                        "Rcut": 5.5,
                        "Resolution": 0.25,
                        "Swtype": "rep",
                        "const_type" : "Mono"
                },
                "Na-Na": {
                        "Rcut": 5.5,
                        "Resolution": 0.25,
                        "Swtype": "rep",
                        "const_type" : "Mono"
                }
        }
}
#SAVE TO FILE
with open('CCS_input.json', 'w') as f:
    json.dump(input, f, indent=8)

In [ ]:
#RUN FIT
from ccs_fit import ccs_fit

ccs_fit("CCS_input.json")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#with open("CCS_params_reference.json", "r") as f:
#    CCS_params_ref = json.load(f)

with open("CCS_params.json", "r") as f:
    CCS_params = json.load(f)

#r_ref = np.array(CCS_params_ref["Two_body"]["Ca-O"]["r"])
#e_ref = CCS_params_ref["Two_body"]["Ca-O"]["spl_a"]


r = np.array(CCS_params["Two_body"]["Cl-Na"]["r"])
e = CCS_params["Two_body"]["Cl-Na"]["spl_a"]
plt.plot(r,e,'--',color='red',label="Cl-Na")

r = np.array(CCS_params["Two_body"]["Na-Na"]["r"])
e = CCS_params["Two_body"]["Na-Na"]["spl_a"]
plt.plot(r,e,'--',color='b',label="Na-Na")

r = np.array(CCS_params["Two_body"]["Cl-Cl"]["r"])
e = CCS_params["Two_body"]["Cl-Cl"]["spl_a"]
plt.plot(r,e,'--',color='g',label="Cl-Cl")

plt.xlim(1.,8)
plt.ylim(-1.5,2.0)
plt.xlabel('Distance (Å)')
plt.ylabel('Potential (eV)')

plt.legend()
plt.show()

# second run with specified switching point.

In [ ]:
### Generate input.json file
import json

input={
    "General": {
        "interface": "CCS+Q",
        "merging"  : "True",
    },
    "Twobody": {
                "Cl-Na": {
                        "Rcut": 5.5,
                        "Resolution": 0.02,
                        "Swtype": "sw",
                        "const_type" : "Mono",
                        "search_mode": "range",
                        "range_center": 2.3,
                        "range_width": 0.2
                },
                "Cl-Cl": {
                        "Rcut": 5.5,
                        "Resolution": 0.02,
                        "Swtype": "rep",
                        "const_type" : "Mono"
                },
                "Na-Na": {
                        "Rcut": 5.5,
                        "Resolution": 0.02,
                        "Swtype": "rep",
                        "const_type" : "Mono"
                }
        }
}
#SAVE TO FILE
with open('CCS_input.json', 'w') as f:
    json.dump(input, f, indent=8)

In [ ]:
from ccs_fit import ccs_fit

ccs_fit("CCS_input.json")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

with open("CCS_params.json", "r") as f:
    CCS_params = json.load(f)


r = np.array(CCS_params["Two_body"]["Cl-Na"]["r"])
e = CCS_params["Two_body"]["Cl-Na"]["spl_a"]
plt.plot(r,e,'--',color='red',label="Cl-Na")

r = np.array(CCS_params["Two_body"]["Na-Na"]["r"])
e = CCS_params["Two_body"]["Na-Na"]["spl_a"]
plt.plot(r,e,'--',color='b',label="Na-Na")

r = np.array(CCS_params["Two_body"]["Cl-Cl"]["r"])
e = CCS_params["Two_body"]["Cl-Cl"]["spl_a"]
plt.plot(r,e,'--',color='g',label="Cl-Cl")

plt.xlim(1.,8)
plt.ylim(-1.5,2.0)
plt.xlabel('Distance (Å)')
plt.ylabel('Potential (eV)')

plt.legend()
plt.show()

# Validation

First, let's see how it performs for the training set



In [ ]:
import ase.db as db
from ccs_fit.ase_calculator.ccs_ase_calculator import CCS
import pandas as pd
with open('CCS_params.json', 'r') as f:
    CCS_params=json.load(f)

df=db.connect('DFT_train.db')
model=[];DFT=[]
for row in df.select():
    struct=row.toatoms()
    nat=len(struct)
    struct.calc=CCS(CCS_params=CCS_params, q=q, charge_scaling=True)
    model.append(struct.get_total_energy()/nat)
    DFT.append(row.energy/nat)
check=pd.DataFrame({'model':model, 'DFT':DFT})

In [ ]:
sns.lmplot(x='model', y='DFT',data=check,fit_reg=True)


In [ ]:
sns.residplot(x='model', y='DFT',data=check)

The same data is already computed by `ccs`

In [ ]:
err=np.loadtxt("CCS_error.out")
plt.xlabel('Reference energy (eV)')
plt.ylabel('Validation energy (eV)')
plt.plot( [min(err[:,0]),max(err[:,0])],[min(err[:,0]),max(err[:,0])],'--',color='black'  )
plt.scatter(err[:,0],err[:,1],facecolors='none', edgecolors='red')
plt.show()
plt.xlabel('Reference energy (eV)')
plt.ylabel('Error in fit (eV)')
plt.scatter(err[:,0],err[:,2],facecolors='none', edgecolors='red')
plt.show()


try:
    err_F=np.loadtxt("CCS_error_forces.out")
    plt.xlabel('Reference force (eV/Å)')
    plt.ylabel('Fitted force (eV/Å)')
    plt.plot( [min(err_F[:,0]),max(err_F[:,0])],[min(err_F[:,0]),max(err_F[:,0])],'--',color='black')
    plt.scatter(err_F[:,0],err_F[:,1],facecolors='none', edgecolors='red',alpha=0.1 )
    plt.show()
except:
    pass



This is for the data we have trained on... what about the data we have not trained on.

In [ ]:
from ase import db
from ccs_fit.ase_calculator.ccs_ase_calculator import CCS
import pandas as pd
with open('CCS_params.json', 'r') as f:
    CCS_params=json.load(f)

db=db.connect('../DFT.db')
model=[];DFT=[]
for row in db.select():
    struct=row.toatoms()
    nat=len(struct)
    struct.calc=CCS(CCS_params=CCS_params, q=q, charge_scaling=True)
    model.append(struct.get_total_energy()/nat)
    DFT.append(row.energy/nat)
check_val=pd.DataFrame({'model':model, 'DFT':DFT})

In [ ]:
sns.lmplot(x='model', y='DFT',data=check,fit_reg=True)
sns.lmplot(x='model', y='DFT',data=check_val,fit_reg=True)


In [ ]:
sns.residplot(x='model', y='DFT',data=check)
sns.residplot(x='model', y='DFT',data=check_val)

Can we do better? probably... try to add more data and rerun.

Export to other formats

In [ ]:
from ccs_fit.scripts.ccs_export_FF import write_FF
write_FF("CCS_params.json")

#MD with ASE

In [ ]:
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin
from ase import units
from ccs_fit.ase_calculator.ccs_ase_calculator import CCS
from ase.io.trajectory import Trajectory
from ase.build import bulk
import json


json_file = open("CCS_params.json")
CCS_params = json.load(json_file)



atoms=bulk('NaCl','rocksalt',a=6.369976562933614,cubic=True)
atoms=atoms*[2,2,2]
charge_dict={"Na":1,"Cl":-1}

calc = CCS(CCS_params=CCS_params)
atoms.calc=calc

print("Initial energy:", atoms.get_potential_energy())

T=1174

# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(atoms, temperature_K=T)

# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = Langevin(atoms, 2 * units.fs, T * units.kB, 0.1)


def printenergy(a):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

#Equillibrate
printenergy(atoms)
for i in range(10):
    dyn.run(100)
    printenergy(atoms)

traj = Trajectory('moldyn.traj', 'w', atoms)
dyn.attach(traj.write, interval=100)


# Now run the dynamics
printenergy(atoms)
for i in range(30):
    print("STEP: ", i*100," ",end='')
    dyn.run(100)
    printenergy(atoms)

print("Final energy:", atoms.get_potential_energy())

In [ ]:
from ase.md.analysis import DiffusionCoefficient
from ase.io.trajectory import Trajectory
traj = Trajectory('moldyn.traj')
Df=DiffusionCoefficient(traj, 200*units.fs, atom_indices=None, molecule=False)
Df.calculate()
D=Df.get_diffusion_coefficients()
conv_factor=units.fs*0.1

print("Na Diffusion constant:", D[0][1]*conv_factor," cm^2/s, Standard deviation",D[1][1]*conv_factor)
print("Cl Diffusion constant:", D[0][0]*conv_factor," cm^2/s, Standard deviation",D[1][0]*conv_factor)


print("REFERENCE VALUES FROM GAP POTENTIALS https://pubs.acs.org/doi/full/10.1021/acs.jpcc.0c08870")

print("Na Diffusion constant:  9.78E-5 cm^2/s, Standard deviation: 0.19E-5 ")
print("Cl Diffusion constant:  7.73E-5 cm^2/s, Standard deviation: 0.11E-5 ")
Df.plot()

In [ ]:
%%capture
!wget https://github.com/Teoroo-CMC/CCS/raw/master/examples/Advanced_Tutorials/MD_tutorial/RefData/ClCl.txt
!wget https://github.com/Teoroo-CMC/CCS/raw/master/examples/Advanced_Tutorials/MD_tutorial/RefData/NaCl.txt
!wget https://github.com/Teoroo-CMC/CCS/raw/master/examples/Advanced_Tutorials/MD_tutorial/RefData/NaNa.txt

In [ ]:
#Radial distribution function

import numpy as np
import itertools as iter
traj = Trajectory('moldyn.traj')

N_bins=60

h_NaNa=np.zeros((N_bins,))
h_NaCl=np.zeros((N_bins,))
h_ClCl=np.zeros((N_bins,))
counter=0
N=len(atoms)

d_NaNa=[]
d_NaCl=[]
d_ClCl=[]

mask_Na=atoms.symbols=='Na'
mask_Cl=atoms.symbols=='Cl'

for a in traj:
    counter += 1
    a.wrap()
    d_all = np.array(a.get_all_distances(mic=True)) # NxN matrix with all pair-distances
    d_NaNa.extend( d_all[mask_Na,:] [:,mask_Na].flatten()) #Extract Na-Na distances
    d_NaCl.extend( d_all[mask_Na,:] [:,mask_Cl].flatten()) #Extract Na-Cl distances
    d_ClCl.extend( d_all[mask_Cl,:] [:,mask_Cl].flatten()) #Extract Cl-Cl distances


hh,r=np.histogram(d_NaNa,range=(0,6),bins=60)
vol=(4/3.)*np.pi*r**3
nrm=np.diff(vol)
h_NaNa = 4*hh/nrm/N/counter
h_NaNa[0]=0

hh,r=np.histogram(d_NaCl,range=(0,6),bins=60)
vol=(4/3.)*np.pi*r**3
nrm=np.diff(vol)
h_NaCl = 4*hh/nrm/N/counter
h_NaCl[0]=0

hh,r=np.histogram(d_ClCl,range=(0,6),bins=60)
vol=(4/3.)*np.pi*r**3
nrm=np.diff(vol)
h_ClCl = 4*hh/nrm/N/counter
h_ClCl[0]=0


import matplotlib.pyplot as plt
Dens=len(atoms)/atoms.get_volume()

plt.ylim(0,4)
plt.plot(r[0:-1],h_ClCl/Dens,label="Cl-Cl",color="green")
plt.plot(r[0:-1],h_NaCl/Dens,label="Na-Cl",color="red")
plt.plot(r[0:-1],h_NaNa/Dens,label="Na-Na",color="blue")


ref_NaNa=np.loadtxt('NaNa.txt')
ref_NaCl=np.loadtxt('NaCl.txt')
ref_ClCl=np.loadtxt('ClCl.txt')

plt.plot(ref_NaNa[:,0],ref_NaNa[:,1,],'--',label="Ref. Na-Na",color="blue")
plt.plot(ref_NaCl[:,0],ref_NaCl[:,1,],'--',label="Ref. Na-Cl",color="red")
plt.plot(ref_ClCl[:,0],ref_ClCl[:,1,],'--',label="Ref. Cl-Cl",color="green")


plt.legend()